# Validation - distribution of optimized AP potentials

This notebook evaluates qualitatively the distributions of membrane potentials and transmembrane currents of an action potential over the neuron morphology.

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
import shutil

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import matplotlib.pyplot as plt
from scipy.spatial import distance
import MEAutility as mu
import json
import time
import numpy as np
from pathlib import Path

import pandas as pd
import seaborn as sns
from scipy.spatial.distance import cosine

import multimodalfitting as mf

%matplotlib notebook

In [ ]:
base_path = Path("../../..")

In [ ]:
abd = True

In [ ]:
colors_dict = {"soma": "C0",
               "all": "C1",
               "sections": "C2",
               "single": "C3"}
figsize = (10, 7)

## Load results and define model

In [ ]:
# general
cell_name = "cell1_211006_3148" # "cell1_211006_3148" | "cell1_211011_3436"
model_name = cell_name
ephys_dir = base_path / "experimental_data" / cell_name / "patch_data"

cell_models_folder = base_path / "cell_models"
model_folder = cell_models_folder / cell_name

In [ ]:
# change this with folder containing your pkl file
results_date = '220214'  # '211124' '220111' # 
result_folder = base_path / "results" / results_date

In [ ]:
pkl_file_name = "runs.pkl"

In [ ]:
data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")
# set strategy column
df_model.loc[:, "strategy"] = df_model["extra_strategy"].values.copy()
df_model.loc[df_model["feature_set"] == "soma", "strategy"] = "soma"

results_name = f"opt_results_training_{model_name}"
if abd:
    results_name += "_abd"
else:
    results_name += "_noabd"
results_file = f"{results_name}.pkl"

opt_results_training = None
if (result_folder / results_file).is_file():
    with open(result_folder / results_file, 'rb') as f:
        opt_results_training = pickle.load(f)

In [ ]:
protocols_file = model_folder / "fitting" / "efeatures" / "protocols_BPO_all.json"
features_file = model_folder / "fitting" / "efeatures" / "features_BPO_all.json"
probe_file = model_folder / "fitting" / "efeatures" / "probe_BPO.json"

In [ ]:
cell = mf.create_experimental_model(model_name=model_name, release=False, abd=abd)
cell_release = mf.create_experimental_model(model_name=model_name, release=True, abd=abd)

probe = mf.define_electrode(probe_file=probe_file)

param_names = [param.name for param in cell.params.values() if not param.frozen]

# params_release = {}
# for param in cell_release.params_by_names(param_names):
#     params_release[param.name] = param.value

In [ ]:
protocol_for_eap = "IDrest_300"
# protocol_for_eap = "firepattern_120"

### Define more recording points 

In [ ]:
extra_kwargs = mf.utils.get_extra_kwargs()

In [ ]:
eva_extra = mf.create_evaluator(
    model_name=model_name,
    strategy="all",
    all_protocols=True,
    protocols_with_lfp=protocol_for_eap,
    **extra_kwargs
)

In [ ]:
mf.plot_cell(eva_extra.cell_model, eva_extra.sim, param_values=params_release, color_ais="g",
             color_axon="b")

In [ ]:
positions = np.array([[270, -113], [-125, 137], [66, -51], [-55, 42],
                     [18.3, -54.6], [28.3, -35.3], [-85, -92], [-306, -13.5]]) 
position_names = ["dend_distal_right", "dend_distal_left", "dend_right_proximal", "dend_left_proximal", 
                  "ais_distal", "ais_proximal", "axon_proximal", "axon_distal"] 

In [ ]:
extra_recordings = mf.utils.extra_recordings_from_positions(cell_release, eva_extra.sim, positions, position_names,
                                                            with_currents=True)

In [ ]:
extra_recordings

In [ ]:
eva_extra = mf.create_evaluator(
    model_name=model_name,
    strategy="all",
    protocols_with_lfp=protocol_for_eap,
    abd=abd,
    all_protocols=True,
    extra_recordings={protocol_for_eap: extra_recordings}, 
    **extra_kwargs
)

# Load protocols and original features

In [ ]:
protocol_to_run = eva_extra.fitness_protocols[protocol_for_eap]
print(protocol_to_run)

In [ ]:
# opt_results = {}
# for strategy in np.unique(df_model.strategy):
#     opt_results[strategy] = {}
#     opt_df = df_model.query(f"strategy == '{strategy}'")
#     best_idx = np.argmin(opt_df.best_fitness)
#     params_sample = opt_df.iloc[best_idx]
#     params_dict = {k: v for k, v in zip(param_names, params_sample.best_params)}
#     opt_results[strategy]["best_fitness"] = params_sample.best_fitness
#     opt_results[strategy]["best_params"] = params_dict
#     print(f"{strategy} --  best fitness: {params_sample.best_fitness}")

In [ ]:
all_strategies = ["soma", "all", "sections", "single"]
strategies = [strategy for strategy in opt_results_training if strategy in all_strategies]

In [ ]:
opt_results = {}
for strategy in strategies:
    print(f"Computing '{strategy}' -- seed: {opt_results_training[strategy]['best_seed']}")
    opt_results[strategy] = {}
    best_params = opt_results_training[strategy]["best_params"]
    t_start = time.time()
    responses = eva_extra.run_protocol(protocol_to_run, best_params)
    t_stop = time.time()
    print(f"Elapsed time: {np.round(t_stop - t_start, 2)}s")
    opt_results[strategy]["responses"] = responses

In [ ]:
strategy = "soma"
print(f"Computing '{strategy}' -- seed: {opt_results_training[strategy]['best_seed']}")
opt_results[strategy] = {}
best_params = opt_results_training[strategy]["best_params"]
t_start = time.time()
responses = eva_extra.run_protocol(protocol_to_run, best_params)
t_stop = time.time()
print(f"Elapsed time: {np.round(t_stop - t_start, 2)}s")
opt_results[strategy]["responses"] = responses

In [ ]:
ms_after = 50
ms_before = 10

In [ ]:
for strategy in strategies:
    print(f"Cutting {strategy}")
    responses = opt_results[strategy]["responses"]
    responses_cut = mf.utils.get_peak_cutout(responses, ms_before=ms_before, 
                                             ms_after=ms_after, average=True)
    opt_results[strategy]["responses_cut"] = responses_cut

In [ ]:
responses_to_plot = []
for resp in responses:
    if any(pos_name in resp for pos_name in position_names):
        responses_to_plot.append(resp)

In [ ]:
print(responses_to_plot)

In [ ]:
xlims = {"dend_distal_right": [-ms_before, ms_after], 
         "dend_distal_left": [-ms_before, ms_after],
         "dend_right_proximal": [-5, 20], 
         "dend_left_proximal": [-5, 20], 
         "ais_distal": [-2, 4],
         "ais_proximal": [-2, 4],
         "axon_proximal": [-2, 5],
         "axon_distal": [-2, 8]}         

In [ ]:
figs_traces = {}

In [ ]:
exclude_strategies = []

In [ ]:
add_title = True

In [ ]:
for strategy in opt_results:
    responses = opt_results[strategy]["responses"]
    mf.plot_responses(responses, color=colors_dict[strategy])

In [ ]:
strategy_arr = []
position_arr = []
for resp_name in responses_to_plot:
    fig, ax = plt.subplots(figsize=figsize)
    position = resp_name.split(".")[1]

    for strategy in opt_results:
        if strategy not in exclude_strategies:
            
            resp_cut = opt_results[strategy]["responses_cut"]
            times = np.linspace(-ms_before, ms_after, len(resp_cut[resp_name]["time"]))
            
            ax.plot(times, resp_cut[resp_name]["voltage"], 
                    color=colors_dict[strategy], label=strategy.upper(), alpha=0.7, lw=1.5)
            position_arr.append(position)
            strategy_arr.append(strategy)
    ax.set_xlabel("time (ms)", fontsize=20)
    ax.set_ylabel("$V_m$ (mV)", fontsize=20)
    ax.set_xlim(xlims[position])
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    if add_title:
        ax.set_title(f"{position}", fontsize=15)
    ax.axvline(0, color="gray", ls="--")
    ax.legend(fontsize=20)
    figs_traces[position] = fig

# df = pd.DataFrame({"strategy": strategy_arr, "distance": distance_arr, "position": position_arr})

In [ ]:
figs_dists = {}

In [ ]:
order = ["soma", "all", "sections", "single"]

In [ ]:
for position in np.unique(df["position"]):
    print(position)
    fig, ax = plt.subplots(figsize=figsize)
    df_pos = df.query(f"position == '{position}'")
    sns.barplot(data=df_pos, x="strategy", y="distance", ax=ax, order=order)
#                 color=[colors_dict[o] for o in order])
    ax.set_xlabel("Strategy", fontsize=20)
    ax.set_ylabel("cos. dist.", fontsize=20)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    if add_title:
        ax.set_title(f"{position}", fontsize=15)
    figs_dists[position] = fig

In [ ]:
fig_cell, ax_cell = plt.subplots(figsize=(7, 10))
mf.plot_cell(eva_extra.cell_model, eva_extra.sim, param_values=params_release, ax=ax_cell,
             alpha=0.5, color="black", detailed=True)

cmap = "tab20"
cm = plt.get_cmap(cmap)
for i, (pos, pos_name) in enumerate(zip(positions, position_names)):
    color = cm(i / len(positions))
    ax_cell.plot(*pos, label=pos_name, marker="o", alpha=0.8, color=color, markersize=3)

In [ ]:
save_fig = True
# figure_folder = Path(".") / "figures_hay_ais"

figure_folder = Path("/Users/abuccino/Documents/Submissions/papers/multimodal/hay_ais/ap_distr")

if save_fig:
    figure_folder.mkdir(exist_ok=True)

In [ ]:
if save_fig:
    fig_cell.savefig(figure_folder / "cell.pdf", transparent=True)
    
    for position in figs_traces:
        figs_traces[position].savefig(figure_folder / f"trace_{position}.pdf", transparent=True)
        figs_dists[position].savefig(figure_folder / f"dist_{position}.pdf", transparent=True)        